In [14]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle

In [4]:
def get_url_in_response(text):
    urls = []
    urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', text)
    return urls

def get_urls(url):
    urls = []
    try:
        r = requests.get(url, verify=False, timeout=2)
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
        urls = get_url_in_response(r.text)
        return urls
    except:
        print("An exception occurred")
        return []


def insert_url_into_arr(urls, new_urls, limit):
    count = 0;
    for url in new_urls:
        if count == limit:
            break
        if url not in urls:
            urls.append(url)
        count += 1
    return urls

def get_urls_recursive(start_url, limit):
    urls = [start_url]
    tmp_list = []
    for url in urls:
        if url not in tmp_list:
            tmp_list.append(url)
            new_urls = get_urls(url)
            if len(new_urls) > 0:
                insert_url_into_arr(urls,new_urls,limit )
            print(len(urls))
        if len(urls) > 200:
            return urls[0:200]
    return urls
url_list = get_urls_recursive('https://www.hcmus.edu.vn', 200)
# print(url_list)

23
36
36
37
48
58
61
69
73
79
83
87
90
91
91
101
113
An exception occurred
113
114
119
120
120
141
141
146
154
174
194
212


In [9]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    r =  None
    try:
        r = requests.get(url)
    except:
        print("An exception occurred")
    if r == None:
        return 
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []
    # print(text)
    translator = str.maketrans('', '', string.punctuation)
    for i in range(len(filtered_text)):
        text_trans = filtered_text[i].translate(translator).strip()
        text_split = text_trans.split(" ")
        for j in range(len(text_split)):
            if len(text_split[j]) != 0:
                word_list.append(text_split[j])
    return ' '.join([str(elem) for elem in word_list])

def read_url(url, url_idx, data):
    content = wordList(url)
    out_dict = dict()
    out_dict["url"] = url
    out_dict["content"] = content
    return out_dict


In [12]:
data = {}
rs = []
for url_index, url in enumerate(url_list, 1):
    res = read_url(url, url_index, data)
    rs.append(res)
    # print(res)

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


In [13]:
import json
with open('data.json', 'w') as fp:
    json.dump(rs, fp,ensure_ascii=False)

In [15]:
# crawler data for search